In [1]:
import pandas as pd
from configurations.general_confs import SparseMethods

In [6]:
def get_dataset(model_name):
    """ Get data set from model name. Is generally in model name for visdom. """
    for d in ["fashionmnist", "cifar10", "mnist"]:
        if d in model_name:
            return d
        
def get_table(best_acc_path, dataset, modelname2method, not_contain_substring=None):
    """ Function to get a table with statistics. 
    
    Args: 
        best_acc_path (str): Path to the best_acc.csv file. 
        dataset (str): Name of the dataset to select as used in the model name. 
        modelname2method (str->str): Function that processes the model name to the method is question. 
        not_contain_substring (str, optional): Substring of the method that should not be in the table. 
            Used to exclude methods.     
    """
    
    # Read the data frame
    df = pd.read_csv(best_acc_path, sep=";")
    df["Score"] = df["Score"] * 100
    
    # retrieve from model name the dataset and add as column
    df["dataset"] = df["ModelName"].apply(lambda name:get_dataset(name))

    # retrieve from model name the sparse method and add as column
    df["method"] = df["ModelName"].apply(lambda x:modelname2method(x))

    # select on the dataset column
    df_sub = df[df['dataset'] == dataset]

    # remove all sparse methods that contain this substring
    if not_contain_substring:
        df_sub = df_sub[~df_sub['method'].str.contains(not_contain_substring)]
    
    # hacky solution to get nice title, rename the method column to the dataset name
    new_method_col = dataset.upper() 
    df_sub = df_sub.rename(index=str, columns={"method": new_method_col})
    
    # get statistics 
    df_sub = df_sub.groupby([new_method_col])["Score"].describe()[["count", "mean", "std"]]
    
    # round to 2 digits 
    df_sub = df_sub.round(2)
    
    return df_sub

def sparse_parser(sparse_str, model_name):
        sparse = SparseMethods(sparse_str)
        
        method_list = []
        for step in sparse:
            if step['target'] == "none":
                method_list.append("none")
            else: 
                percent_list = '-'.join([str(e) for e in step['percent']])
                method_list.append(f"{step['target']} {step['method']} {percent_list}")
        
        sparse_name = " & ".join(method_list)
        
        if "no" in model_name[-2:]: 
            sparse_name += " No"
        
        return sparse_name

## Diff sparse experiment


Define function to get sparse method from model name

In [7]:
def get_sparse_method(model_name):
    """ Get sparse method from model name. 
    
    Model names have to be formatted as: {dataset}_:{sparse_method}:

    Example: cifar10_:edges_random_0.3-0.3:
    """
    
    if "loss" in model_name: 
        if "loss_001" in model_name:
            sparse_name = "Entropy (0.01)"
        elif "loss_01" in model_name:
            sparse_name = "Entropy (0.1)"
        
    else: 
        # get sparse string from model_name
        sparse_str = model_name.split(":")[1]

        sparse_name = sparse_parser(sparse_str, model_name)
            
    return sparse_name        
    

In [8]:
get_table("./experiments/diff_sparse2/best_acc.csv", "cifar10", get_sparse_method, " No")

,count,mean,std
CIFAR10,,,
Entropy (0.01),8.0,65.56,0.90
Entropy (0.1),8.0,62.22,3.62
edges random 0.0-0.3,4.0,64.32,0.28
edges random 0.0-0.6,4.0,64.11,0.16
edges random 0.3-0.3,4.0,63.91,0.92
edges sample 0.3-0.3,4.0,63.96,1.08
edges topk 0.3-0.3,4.0,64.35,0.81
nodes random 0.2-0.2 & edges random 0.2-0.2,4.0,43.37,0.90
nodes random 0.3-0.3,4.0,38.53,0.63


## Routing effect

In [13]:
def get_routing_iters_used(model_name):
    """ Get Number of routing iterations from model name. 
    
    Model names have to be formatted as: {dataset}_:{routing_iters}:

    Example: cifar10_:1:
    """
    split = model_name.split(":")
    sparse_method = sparse_parser(split[1], model_name)
    routing_iters = split[2]
    arch = split[4]
    
    
    return f"{sparse_method} {routing_iters} {arch}"

In [17]:
get_table("./experiments/effect_routing/best_acc.csv", "fashionmnist", get_routing_iters_used, "double")

,count,mean,std
FASHIONMNIST,,,
none 1 single,3.0,91.13,0.09
none 2 single,3.0,90.35,0.21
none 3 single,3.0,90.06,0.23
